In [1]:
# 주로 사용하는 코드 2 : 인식한 GPU 개수 출력
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
df= pd.read_csv('./datasets_tf/creditcard.csv', na_values='#NAME?')
# 데이터 파일에서 #NAME? 값이 있으면 결즉치(값 비어있거나 존재하지 않는 경우)로 처리
# 앞에 점을 찍어줘야 현재 작업 디렉토리 라는 뜻으로 이해 가능함

x=df[['V17','V9','V6','V12']]
y=df['Class']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3, random_state=101)
#random state은 동일 시드를 생성하는 것이다. 

In [4]:
print(y)

0         0
1         0
2         0
3         0
4         0
         ..
284802    0
284803    0
284804    0
284805    0
284806    0
Name: Class, Length: 284807, dtype: int64


In [5]:
print(x)

             V17        V9        V6       V12
0       0.207971  0.363787  0.462388 -0.617801
1      -0.114805 -0.255425 -0.082361  1.065235
2       1.109969 -1.514654  1.800499  0.066084
3      -0.684093 -1.387024  1.247203  0.178228
4      -0.237033  0.817739  0.095921  0.538196
...          ...       ...       ...       ...
284802  1.991691  1.914428 -2.606837  2.711941
284803 -0.025693  0.584800  1.058415  0.915802
284804  0.313502  0.432454  3.031260  0.063119
284805  0.509928  0.392087  0.623708 -0.962886
284806 -0.660377  0.486180 -0.649617 -0.031513

[284807 rows x 4 columns]


In [12]:
# Random search
# 구간 정해주고 그 구간 안 랜덤으로 숫자 뽑아서 실험

from sklearn.model_selection import RandomizedSearchCV
#트리의 노드를 반복적으로 분할

"""
criterion: 노드 분할 기준 정하는 매개변수, entropy/ gini 중 택일
max_depth: 트리의 최대깊이 제한 과적합 방지
max_features: 특성 개수의 제곱근 만큼 특성 선택
min_samples_leaf: 리프 노드가 되기 위한 최소 샘플 수 지정, 이 값보다 작으면 더이상
                    분할하지 않고 리프 노드가 된다
min_samples_split: 노드를 분할하기 위한 최소 샘플 수 지정, 이 값보다 작으면 더이상 
                    분할 하지 않고 리프 노드 됨
n_estimators: 생성할 트리의 개수 지정"""

random_search={'criterion':['entropy','gini'],
              'max_depth':[2],
              'max_features':['auto','sqrt'],
              'min_samples_leaf':[4,6,8],
              'min_samples_split':[5,7,10],
              'n_estimators':[20]}

clf=RandomForestClassifier()
model=RandomizedSearchCV(estimator=clf,param_distributions=random_search,n_iter=10,
                        cv=4, verbose=1,random_state=101,n_jobs=-1)
"""
estimator: 탐색할 모델
param_distributions: 탐색할 하이퍼파라미터 공간, 이 공간에서 무작위 샘플링
n_iter: 랜덤 탐색에서 시도할 하이퍼파라미터 조합의 수
cv: 교차검정을 수행할 폴드 수
n_jobs: 적합성과 예측성을 위해 병렬로 실행할 작업 수
        병렬 처리에 사용할 cpu 코어의 수
        -1로 설정 시 가능한 모든 코어 사용"""
model.fit(x_train,y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [2],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [4, 6, 8],
                                        'min_samples_split': [5, 7, 10],
                                        'n_estimators': [20]},
                   random_state=101, verbose=1)

In [13]:
print(model.best_params_)

{'n_estimators': 20, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 2, 'criterion': 'gini'}


In [15]:
randompredict=model.best_estimator_.predict(x_test)
print(classification_report(y_test,randompredict))
print("\nRandom Search: ", accuracy_score(y_test,randompredict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85299
           1       0.84      0.56      0.67       144

    accuracy                           1.00     85443
   macro avg       0.92      0.78      0.83     85443
weighted avg       1.00      1.00      1.00     85443


Random Search:  0.9990754069964772


In [16]:
#grid search
"""
random search와 차이점
random: 탐색 범위 주어지고 해당 범위 내 임의 조합 찾아 탐색
grid: 탐색할 값이 주어지고 모든 조합 탐색
"""

from sklearn.model_selection import GridSearchCV

grid_search={'criterion':['entropy','gini'],
            'max_depth':[2],
            'max_features':['auto','sqrt'],
            'min_samples_leaf':[4,6,8],
            'min_samples_split':[5,7,10],
            'n_estimators':[20]}
clf=RandomForestClassifier()
model=GridSearchCV(estimator=clf, param_grid=grid_search,cv=4,verbose=5,n_jobs=-1)
model.fit(x_train,y_train)

Fitting 4 folds for each of 36 candidates, totalling 144 fits


GridSearchCV(cv=4, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy', 'gini'], 'max_depth': [2],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [4, 6, 8],
                         'min_samples_split': [5, 7, 10],
                         'n_estimators': [20]},
             verbose=5)

In [17]:
print(model.best_params_)

{'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 20}


In [18]:
gridpredict=model.best_estimator_.predict(x_test)
print(classification_report(y_test,gridpredict))
print("\nGrid Search: ",accuracy_score(y_test,gridpredict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85299
           1       0.83      0.69      0.76       144

    accuracy                           1.00     85443
   macro avg       0.92      0.85      0.88     85443
weighted avg       1.00      1.00      1.00     85443


Grid Search:  0.9992509626300574


In [19]:
# Bayesian Optimization
"""
1. 관측 데이터 기반 목적 함수를 추정: Gaussian Process
2. 추정 모델 기반 탐색한 파라미터 선택: Acquisition Function
-평균이 최대
-분산이 최대
3. 다음 관측 데이터에 추가

목적함수와 하이퍼 파라미터 조합을 대상으로 평가 후 관측 데이터에 추가하는 과정을 
반복하면서 순차적으로 업데이트 하여 최적의 조합을 탐색하는 방법"""

"""
입력값 x 받는 미지의 목적 함수 f(x) 상정 후 해당 함숫값을 최대로 만드는 최적해를
찾는 것을 목적으로 한다. 
두가지 필수 요소 존재
1. Surrogate Model은 현재까지 조사된 입력값-함숫값 점 바탕으로 미지의 목적 함수 형태
에 대한 확률적인 추정을 수행하는 모델
2. Acquisition Function은 목적 함수에 대한 현재까지의 확률적 추정 결과를 바탕으로
최적 입력값을 찾는 데 있어 가장 유용할 만한 다음 입력값 후보를 추천해 주는 함수 지칭
"""

!pip install hyperopt

     ---------------------------------------- 1.6/1.6 MB 10.1 MB/s eta 0:00:00
     ------------------------------------- 840.9/840.9 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 78.3/78.3 kB 4.3 MB/s eta 0:00:00
     -------------------------------------- 200.5/200.5 kB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 11.1 MB/s eta 0:00:00
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492055 sha256=24695b4636231947cf6483ed6a7e6e98e03b4fbf168cdb8527c3197fcee56e93
  Stored in directory: c:\users\tlsdu\appdata\local\pip\cache\wheels\52\2a\fc\520209cfa6448febd490720a0b09036cb367628f7c4e9cc172
Successfully built future


In [21]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

space={'criterion':hp.choice('criterion',['entropy','gini']),
      'max_depth':hp.quniform('max_depth',10,12,10),
      'max_features':hp.choice('max_features',['auto','sqrt','log2',None]),
      'min_samples_leaf':hp.uniform('min_samples_leaf',0,0.5),
      'min_samples_split':hp.uniform('min_samples_split',0,1),
      'n_estimators':hp.choice('n_estimators',[10,50])}
#hp.quniform 은 실수 값 반환하지만 주어진 간격에 따라 양자화된다
# 10 부터 12까지의 값 균일하게 샘플링 하지만 반환된 값은 10,11,12 중 하나일 것이다.

def objective(space):
    hopt=RandomForestClassifier(criterion=space['criterion'],
                               max_depth=space['max_depth'],
                               max_features=space['max_features'],
                               min_samples_leaf=space['min_samples_leaf'],
                               min_samples_split=space['min_samples_split'],
                               n_estimators=space['n_estimators'])
    accuracy=cross_val_score(hopt,x_train,y_train,cv=4).mean()
    return {'loss':accuracy,'status':STATUS_OK}

trials=Trials()
best=fmin(fn=objective,
         space=space,
         algo=tpe.suggest,
         max_evals=20,
         trials=trials)

best

ERROR! Session/line number was not unique in          
database. History logging moved to new session        
209                                                   
100%|██████████| 20/20 [01:53<00:00,  5.66s/trial, best loss: 0.9982544491482915]


{'criterion': 0,
 'max_depth': 10.0,
 'max_features': 3,
 'min_samples_leaf': 0.10591490048784225,
 'min_samples_split': 0.21029259237553422,
 'n_estimators': 1}

In [22]:
crit={0:'entropy',1:'gini'}
feat={0:'auto',1:'sqrt',2:'log2',3:None}
est={0:10,1:50,2:75,3:100,4:125}

trainedforest=RandomForestClassifier(criterion=crit[best['criterion']],
                                    max_depth=best['max_depth'],
                                    max_features=feat[best['max_features']],
                                    min_samples_leaf=best['min_samples_leaf'],
                                    min_samples_split=best['min_samples_split'],
                                    n_estimators=est[best['n_estimators']]
                                    ).fit(x_train,y_train)

predictionforest=trainedforest.predict(x_test)
print(classification_report(y_test,predictionforest))

ERROR! Session/line number was not unique in database. History logging moved to new session 212
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85299
           1       0.00      0.00      0.00       144

    accuracy                           1.00     85443
   macro avg       0.50      0.50      0.50     85443
weighted avg       1.00      1.00      1.00     85443



C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print("Bayesian Optimization: ",accuracy_score(y_test,predictionforest))

Bayesian Optimization:  0.9983146659176293


In [24]:
!pip install optuna

     -------------------------------------- 413.4/413.4 kB 8.6 MB/s eta 0:00:00
     ------------------------------------- 226.8/226.8 kB 14.4 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 11.0 MB/s eta 0:00:00
     -------------------------------------- 153.2/153.2 kB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB ? eta 0:00:00
     ------------------------------------- 291.4/291.4 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0


In [ ]:
import optuna
import sklearn
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm

def objective(trial):
    iris=sklearn.datasets.load_iris()
    x,y=iris.dat,iris.target
    
    classifier_name=trai.suggest_categorical('classifier',['SVC','RandomForest'])

In [2]:
pip install scikit-optimize

     ---------------------------------------- 100.3/100.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Preprocess data
X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



# Define search space for hyperparameters
search_space = {
    'learning_rate': Real(0.0001, 0.1, prior='log-uniform'),
    'num_filters': Integer(16, 64),
    'kernel_size': Integer(3, 5),
    'dropout_rate': Real(0.0, 0.5)
}

# Initialize Bayesian Optimization
bayes_cv_tuner = BayesSearchCV(
    estimator=create_model(),
    search_spaces=search_space,
    n_iter=20,
    cv=3,
    n_jobs=-1,
    verbose=1
)
# Define CNN model
def create_model(learning_rate, num_filters, kernel_size, dropout_rate):
    model = Sequential([
        Conv2D(num_filters, kernel_size, activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(num_filters * 2, kernel_size, activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Fit model with Bayesian Optimization
result = bayes_cv_tuner.fit(X_train, y_train)

# Get best hyperparameters and evaluate on test set
best_params = result.best_params_
best_model = result.best_estimator_
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


ERROR! Session/line number was not unique in database. History logging moved to new session 243


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tlsdu\AppData\Local\Temp\ipykernel_12652\262957053.py", line 32, in <module>
    estimator=create_model(),
TypeError: create_model() missing 4 required positional arguments: 'learning_rate', 'num_filters', 'kernel_size', and 'dropout_rate'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\IPython\core\ultratb.py", line

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Preprocess data
X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define search space for hyperparameters
search_space = {
    'learning_rate': Real(0.0001, 0.1, prior='log-uniform'),
    'num_filters': Integer(16, 64),
    'kernel_size': Integer(3, 5),
    'dropout_rate': Real(0.0, 0.5)
}

# Define CNN model
def create_model(learning_rate, num_filters, kernel_size, dropout_rate):
    model = Sequential([
        Conv2D(num_filters, kernel_size, activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(num_filters * 2, kernel_size, activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize Bayesian Optimization
bayes_cv_tuner = BayesSearchCV(
    estimator=create_model,
    search_spaces=search_space,
    n_iter=20,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit model with Bayesian Optimization
result = bayes_cv_tuner.fit(X_train, y_train)

# Get best hyperparameters and evaluate on test set
best_params = result.best_params_
best_model = result.best_estimator_
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


ERROR! Session/line number was not unique in database. History logging moved to new session 246


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tlsdu\AppData\Local\Temp\ipykernel_15036\3922099276.py", line 55, in <module>
    result = bayes_cv_tuner.fit(X_train, y_train)
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\skopt\searchcv.py", line 466, in fit
    super().fit(X=X, y=y, groups=groups, **fit_params)
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\sklearn\model_selection\_search.py", line 793, in fit
    scorers = check_scoring(self.estimator, self.scoring)
  File "C:\Users\tlsdu\anaconda3\envs\mine\lib\site-packages\sklearn\metrics\_scorer.py", line 450, in check_scoring
    % estimator
TypeError: estimator should be an estimator implementing 'fit' method, <function create_model at 0x000002029D41A678> was passed

During handling of the above exception, another exception o

In [1]:
pip install bayesian-optimization

ERROR! Session/line number was not unique in database. History logging moved to new session 248
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from bayes_opt import BayesianOptimization


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)


In [4]:
def build_model(lr=1e-3, dropout_rate=0.5):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    base_model.trainable = False  # 기본 모델의 가중치를 고정합니다.

    model = Sequential([
        base_model,
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [5]:
def fit_with(lr, dropout_rate):
    model = build_model(lr, dropout_rate)
    history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=0)
    accuracy = np.amax(history.history['val_accuracy'])  # 검증 정확도의 최댓값을 반환합니다.
    return accuracy


In [6]:
pbounds = {'lr': (1e-4, 1e-2), 'dropout_rate': (0.1, 0.5)}

optimizer = BayesianOptimization(
    f=fit_with,
    pbounds=pbounds,
    random_state=1,
)
optimizer.maximize(init_points=2, n_iter=10)


|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------
58892288/58889256 [==============================] - 6s 0us/step
| 1         | 0.5855    | 0.2668    | 0.007231  |
| 2         | 0.605     | 0.1       | 0.003093  |
| 3         | 0.6041    | 0.1435    | 0.003752  |
| 4         | 0.6011    | 0.1436    | 0.003685  |
| 5         | 0.5975    | 0.143     | 0.003873  |
| 6         | 0.6063    | 0.1003    | 0.003499  |
| 7         | 0.6251    | 0.2583    | 0.001698  |
| 8         | 0.5848    | 0.4551    | 0.004527  |
| 9         | 0.602     | 0.2729    | 0.003585  |
| 10        | 0.6185    | 0.258     | 0.002004  |
| 11        | 0.6058    | 0.1002    | 0.003121  |
| 12        | 0.5979    | 0.1433    | 0.003671  |
